In [9]:
import pandas as pd
import logging

# Log fayl manzili
log_path = r"C:\Users\Rasulbek907\Desktop\Project_8\Log\error_analysis.log"

# Log sozlamalari
logging.basicConfig(
    filename=log_path,
    filemode='a',  # Append mode
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO
)

try:
    logging.info("CSV fayl o'qilmoqda:...")
    df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_8\Data\Feature_Selection\Filtered_Features.csv")
    logging.info(f"Fayl muvaffaqiyatli o'qildi. Satırlar soni: {len(df)} ustunlar soni: {len(df.columns)}")
except Exception as e:
    logging.error(f"CSV faylni o'qishda xatolik: {e}")

In [ ]:
# 1️⃣ SHAP sample tayyorlash (tezkor ishlash uchun)
X_shap_sample = X_shap_df.sample(n=2000, random_state=42)
y_sample = y_test.loc[X_shap_sample.index]  # agar kerak bo‘lsa

# 2️⃣ TreeExplainer
explainer = shap.TreeExplainer(gb_model)
shap_values_sample = explainer.shap_values(X_shap_sample)

# 3️⃣ Dashboard funksiyasi (faqat sample subset)
def shap_dashboard_sample(shap_values, X_shap_df, explainer):
    feature_dropdown = widgets.Dropdown(
        options=X_shap_df.columns.tolist(),
        description='Feature:',
        disabled=False,
    )
    sample_slider = widgets.IntSlider(
        value=0,
        min=0,
        max=len(X_shap_df)-1,
        step=1,
        description='Sample:',
        continuous_update=False
    )
    out = widgets.Output()

    def update_plot(change):
        with out:
            clear_output(wait=True)
            # Summary plots
            shap.summary_plot(shap_values.values, X_shap_df, plot_type="dot", show=True)
            shap.summary_plot(shap_values.values, X_shap_df, plot_type="bar", show=True)
            # Dependence plot
            shap.dependence_plot(feature_dropdown.value, shap_values.values, X_shap_df, show=True)
            # Force plot
            shap.initjs()
            display(shap.force_plot(explainer.expected_value, shap_values.values[sample_slider.value,:], X_shap_df.iloc[sample_slider.value,:]))
            # Waterfall plot
            shap.plots._waterfall.waterfall_legacy(explainer.expected_value, shap_values.values[sample_slider.value,:], feature_names=X_shap_df.columns)
            # Decision plot (first 100 samples)
            shap.decision_plot(explainer.expected_value, shap_values.values[:100], X_shap_df.iloc[:100,:], show=True)

    feature_dropdown.observe(update_plot, names='value')
    sample_slider.observe(update_plot, names='value')
    display(widgets.VBox([feature_dropdown, sample_slider, out]))
    update_plot(None)

# 4️⃣ Dashboard ishga tushirish
shap_dashboard_sample(shap_values_sample, X_shap_sample, explainer)
